# Übung 4 | Planeten- und Satellitenbahnen (bepohl@ethz.ch)

In [1]:
import numpy as np
from tqdm import tqdm
import datetime

## Aufgabe 1

M + m = ((2pi) / T )^2 * a^3 / G

In [2]:
T_moon = 27 * 86400 # seconds
H_moon = 400000 * 10**3 # meters
G = 6.673 * 10**(-11) # m^3/(kg*s^2)

M_ges = ((2*np.pi / T_moon)**2) * (H_moon**3 / G)

print("Masse von Erde-Mond-System:", "{:.3e}".format(M_ges), "kg")

Masse von Erde-Mond-System: 6.958e+24 kg


## Aufgabe 2

In [15]:
T_s = 8720 # seconds
c_s = 9.448 * 10**9 # m^2/s
GM = 3.986 * 10**14 # m^3/s^2

a = (T_s**2 * GM / (4 * np.pi**2))**(1/3)
b = c_s * T_s / (a * np.pi)

print("Kleine Halbachse:", round(a / 10**3, 3), "km, Grosse Halbachse:", round(b / 10**3, 3), "km")

Kleine Halbachse: 9156.655 km, Grosse Halbachse: 2863.978 km


In [4]:
t_1 = 2 * 3600 + 30 * 60
t_0 = 2 * 3600 + 20 * 60

M_t1 = 2 * np.pi / T_s * (t_1 - t_0)
e = (1 - b**2 / a**2)**(1/2)

E = M_t1
for n in tqdm(range(10)):
    E = M_t1 + e * np.sin(E)

v = 2 * np.arctan(np.tan(E / 2) * np.sqrt((1 + e) / (1 - e)))

100%|██████████| 10/10 [00:00<00:00, 74764.78it/s]


In [21]:
p = a * (1 - e**2)

r = p / (1 + e * np.cos(v))

x = r * np.cos(v)
y = r * np.sin(v)

#V_s = np.sqrt( (GM * np.sin(v) / p)**2 + (GM * (e + np.cos(v)) / p)**2 )
V_s = np.sqrt( (-np.sqrt(GM / p) * np.sin(v))**2 + (np.sqrt(GM / p) * (e + np.cos(v)))**2 )

print("X:", x, "m, Y:", y, "m, Velocity (I think it's right now):", V_s, "m/s")

X: -6793939.802508924 m, Y: 2801424.757128903 m, Velocity (I think it's right now): 8059.055429358593 m/s


## Aufgabe 3

In [6]:
e_3 = 0.3
v_3 = 51 * np.pi / 180 + 30 * (np.pi / (60 * 180)) + 36 * np.pi / (3600 * 180)
t_3 = 16 * 3600 + 45 * 60 + 50
T_3 = 2 * 3600 + 30 * 60 + 20


E_t3 = 2 * np.arctan( np.tan(v_3 / 2) * np.sqrt( (1 - e_3) / (1 + e_3)) )
M_t3 = E_t3 - e_3 * np.sin(E_t3)

t_3_peri_alt = t_3 - T_3 * M_t3 / (2 * np.pi)

t_3_apo = t_3_peri_alt + T_3 / 2
t_3_peri = t_3_peri_alt + T_3

print("Perigäum Vergangenheit:", str(datetime.timedelta(seconds=t_3_peri_alt)))
print("Apogäum Zukunft:", str(datetime.timedelta(seconds=t_3_apo)))
print("Perigäum Zukunft:", str(datetime.timedelta(seconds=t_3_peri)))

Perigäum Vergangenheit: 16:34:04.054246
Apogäum Zukunft: 17:49:14.054246
Perigäum Zukunft: 19:04:24.054246


## Aufgabe 4

In [12]:
OMEGA = 7.29 * 10**-5 # rad / s
R_e = 6378 * 10**3 # m

H_sat = ((GM / OMEGA**2))**(1/3) - R_e
print("Höhe des Satelliten:", round((H_sat / 10**3), 2), "km")

Höhe des Satelliten: 35794.31 km
